# **Minor Changes to Evaluate Ensembles -**

In [ ]:
! pip install -q tensorflow numpy
! cp drive/My\ Drive/Colab\ Notebooks/*.csv ./
! cp drive/My\ Drive/Colab\ Notebooks/*.pkl ./
! cp -r drive/My\ Drive/Colab\ Notebooks/models/ensemble ./

In [ ]:
def ohe(map):
  one_hot_encode = []
  temp = df[['Visual']].values

  for i in temp:
    arr = list(np.zeros(len(map),dtype=int))
    for j in i[0].split():
      if j.strip(',') != '0':
        arr[map[j.strip(',')]] = 1
    one_hot_encode.append(arr)
  return np.array(one_hot_encode)

In [ ]:
def create_map():
  temp = df[['Visual']].values
  counter = 0
  map = {}
  for i in temp:
    for j in i[0].split():
      if j.strip(',') not in map and j.strip(',') != '0':
        map[j.strip(',')] = counter
        counter += 1
  return map

In [ ]:
def evaluate(model, X, y):
  kfold = StratifiedKFold(n_splits=5).split(X, y)
  scores = []
  for k, (train, test) in enumerate(kfold):
    score = model.evaluate(X[test], y[test], verbose=0)
    print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))
    scores.append(score[1] * 100)
    scoring((model(X[test]) > 0.5).numpy().astype("int32").tolist(), y[test])
  print("%.2f%% (+/- %.2f%%)" % (np.mean(scores), np.std(scores)))

In [ ]:
def scoring(predictions, y_test):
  fp = 0
  tp = 0
  fn = 0 
  c = 0
  incorrect = []

  for pred, test in zip(predictions, y_test):
    if pred[0] != test[0]:
      incorrect.append(c)
    if pred[0] == 1:
      if pred[0] == test[0]:
        tp += 1
      else:
        fp += 1
    elif pred[0] != test[0]:
      fn += 1
    c += 1

  print(tp, fp, fn)

  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1 = 2 * precision * recall / (precision + recall)
  print('Precision: ', precision, '\nRecall: ', recall, '\nF1 Score: ', f1)

In [ ]:
from tensorflow import keras

alex = keras.models.load_model('ensemble/alexnet_ffwd')
dense = keras.models.load_model('ensemble/densenet_ffwd')
resnet = keras.models.load_model('ensemble/resnet_ffwd')

In [ ]:
import pandas as pd

df = pd.read_csv('dankmemes_task1_train.csv')
embedding = pd.read_csv('dankmemes_task1_train_embeddings.csv', header=None)

In [ ]:
import numpy as np
import pickle

des_X, alex_X, res_X, y = pickle.load(open('densenest_embeddings.pkl', 'rb')), pickle.load(open('alexnet_embeddings.pkl', 'rb')), np.array([embedding[1][i].split() for i in range(1600)]).astype(float), df[['Meme']].values

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder

ssc = StandardScaler()
mms = MinMaxScaler()

In [ ]:
import pickle
from datetime import date

temp = mms.fit_transform(np.array([(date(int(i[0].split('-')[0]), int(i[0].split('-')[1]), int(i[0].split('-')[2])) - date(2015, 1, 1)).days for i in df[['Date']].values.tolist()]).reshape(1600, 1))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ssc.fit_transform(df[['Engagement']].values)
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = ohe(create_map())
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

temp = pickle.load(open('cls_umberto_embeddings.pkl', 'rb'))
print(temp.shape)
des_X = np.hstack((des_X, temp))
alex_X = np.hstack((alex_X, temp))
res_X = np.hstack((res_X, temp))
print(des_X.shape)
print(alex_X.shape)
print(res_X.shape)

(1600, 1)
(1600, 1001)
(1600, 4097)
(1600, 2049)
(1600, 1)
(1600, 1002)
(1600, 4098)
(1600, 2050)
(1600, 71)
(1600, 1073)
(1600, 4169)
(1600, 2121)
(1600, 768)
(1600, 1841)
(1600, 4937)
(1600, 2889)


In [ ]:
## Suffle Dataframe

seed = np.random.randint(0, 100000)  

np.random.seed(seed)
np.random.shuffle(des_X)
np.random.seed(seed)
np.random.shuffle(alex_X)
np.random.seed(seed)
np.random.shuffle(res_X)
np.random.seed(seed)
np.random.shuffle(y)

In [ ]:
## Individual Scores

import numpy as np
from sklearn.model_selection import StratifiedKFold

evaluate(alex, alex_X, y)
evaluate(dense, des_X, y)
evaluate(resnet, res_X, y)

In [ ]:
kfold = StratifiedKFold(n_splits=5).split(alex_X, y)
a_sc = []
d_sc = []
r_sc = []

for k, (train, test) in enumerate(kfold):
  a = alex.evaluate(alex_X[test], y[test], verbose=0)
  d = dense.evaluate(des_X[test], y[test], verbose=0)
  r = resnet.evaluate(res_X[test], y[test], verbose=0)
  print("%s: %.2f%%" % (alex.metrics_names[1], a[1]*100))
  print("%s: %.2f%%" % (dense.metrics_names[1], d[1]*100))
  print("%s: %.2f%%" % (resnet.metrics_names[1], r[1]*100))
  a_sc.append(a[1] * 100)
  d_sc.append(d[1] * 100)
  r_sc.append(r[1] * 100)

  final = []
  a = (alex(alex_X[test]) > 0.5).numpy().astype("int32").tolist()
  d = (dense(des_X[test]) > 0.5).numpy().astype("int32").tolist()
  r = (resnet(res_X[test]) > 0.5).numpy().astype("int32").tolist()
  for i in range(len(a)):
    # print(a[i][0] + d[i][0] + r[i][0])
    if a[i][0] + d[i][0] + r[i][0] >= 2:
      final.append(1)
    else:
      final.append(0)
  final = np.array(final)
  final = final.reshape(final.shape[0], 1)
  print(final.shape)
  scoring(final, y[test])
print("%.2f%% (+/- %.2f%%)" % (np.mean(a_sc), np.std(a_sc)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(d_sc), np.std(d_sc)))
print("%.2f%% (+/- %.2f%%)" % (np.mean(r_sc), np.std(r_sc)))

binary_accuracy: 94.69%
binary_accuracy: 90.31%
binary_accuracy: 89.38%
(320, 1)
150 6 10
Precision:  0.9615384615384616 
Recall:  0.9375 
F1 Score:  0.9493670886075949
binary_accuracy: 93.75%
binary_accuracy: 90.00%
binary_accuracy: 89.38%
(320, 1)
147 7 12
Precision:  0.9545454545454546 
Recall:  0.9245283018867925 
F1 Score:  0.939297124600639
binary_accuracy: 92.19%
binary_accuracy: 88.44%
binary_accuracy: 85.31%
(320, 1)
147 11 12
Precision:  0.930379746835443 
Recall:  0.9245283018867925 
F1 Score:  0.9274447949526813
binary_accuracy: 91.87%
binary_accuracy: 87.81%
binary_accuracy: 87.81%
(320, 1)
143 9 16
Precision:  0.9407894736842105 
Recall:  0.89937106918239 
F1 Score:  0.9196141479099678
binary_accuracy: 91.56%
binary_accuracy: 87.50%
binary_accuracy: 82.50%
(320, 1)
147 16 12
Precision:  0.901840490797546 
Recall:  0.9245283018867925 
F1 Score:  0.9130434782608695
92.81% (+/- 1.20%)
88.81% (+/- 1.14%)
86.88% (+/- 2.64%)


In [ ]:
### Cumulative Score

a = (alex(alex_X) > 0.5).numpy().astype("int32").tolist()
d = (dense(des_X) > 0.5).numpy().astype("int32").tolist()
r = (resnet(res_X) > 0.5).numpy().astype("int32").tolist()

final = []
for i in range(len(a)):
  # print(a[i][0] + d[i][0] + r[i][0])
  if a[i][0] + d[i][0] + r[i][0] >= 2:
    final.append(1)
  else:
    final.append(0)
final = np.array(final)
final = final.reshape(final.shape[0], 1)
print(final.shape)

(1600, 1)


In [ ]:
scoring(final, y)

734 49 62
Precision:  0.9374201787994891 
Recall:  0.9221105527638191 
F1 Score:  0.9297023432552248


In [ ]:
## The following are the images the ensemble got wrong; note it's shuffled from original!
c = 0 
for i in range(len(final)):
  if final[i][0] == y[i][0]:
    c += 1
  else:
    print(i, '-', final[i][0], y[i][0])
print('\nAccuracy: ' + str(c/16) + '%')

15 - 0 1
18 - 0 1
53 - 0 1
59 - 1 0
100 - 1 0
119 - 1 0
120 - 0 1
146 - 1 0
192 - 1 0
193 - 1 0
215 - 0 1
238 - 0 1
268 - 0 1
269 - 0 1
271 - 0 1
292 - 0 1
356 - 0 1
372 - 1 0
381 - 1 0
390 - 0 1
393 - 1 0
431 - 0 1
436 - 0 1
447 - 0 1
460 - 1 0
461 - 1 0
467 - 1 0
469 - 0 1
495 - 0 1
498 - 0 1
519 - 0 1
541 - 0 1
579 - 1 0
590 - 0 1
597 - 0 1
656 - 0 1
672 - 1 0
676 - 0 1
682 - 0 1
689 - 1 0
693 - 1 0
694 - 1 0
696 - 1 0
711 - 1 0
715 - 0 1
742 - 0 1
744 - 1 0
779 - 0 1
788 - 0 1
790 - 0 1
837 - 0 1
854 - 1 0
869 - 0 1
889 - 1 0
892 - 1 0
929 - 0 1
935 - 1 0
946 - 0 1
967 - 1 0
976 - 1 0
983 - 0 1
987 - 0 1
1002 - 0 1
1011 - 0 1
1012 - 1 0
1023 - 0 1
1056 - 0 1
1059 - 1 0
1068 - 0 1
1073 - 0 1
1084 - 0 1
1100 - 1 0
1116 - 0 1
1121 - 1 0
1140 - 0 1
1170 - 0 1
1176 - 1 0
1192 - 1 0
1195 - 0 1
1230 - 0 1
1242 - 1 0
1247 - 0 1
1255 - 0 1
1287 - 1 0
1301 - 1 0
1309 - 1 0
1315 - 1 0
1318 - 1 0
1325 - 0 1
1332 - 0 1
1341 - 0 1
1367 - 1 0
1373 - 0 1
1413 - 1 0
1428 - 0 1
1446 - 1 0
1465 - 1 0

In [ ]:
X_inc = []

for i, val in enumerate(X):
  for j in incorrect:
    if np.all(X_test[j][:1000] == val[:1000]):
      X_inc.append(i)

In [ ]:
print(X_inc) # Incorrectly classified representative to dataset

[22, 125, 138, 256, 260, 265, 294, 311, 332, 417, 450, 456, 462, 489, 502, 504, 520, 533, 536, 545, 566, 589, 609, 613, 622, 629, 652, 700, 708, 742, 803, 840, 851, 868, 891, 911, 993, 1018, 1058, 1071, 1072, 1104, 1121, 1163, 1172, 1219, 1224, 1240, 1266, 1336, 1339, 1357, 1373, 1403, 1448, 1470, 1472, 1509, 1513, 1519, 1588, 1598]
